#### Import necessary libraries

In [73]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

#### Load Data

In [74]:
df_links=pd.read_csv('Data/links.csv')
df_movies=pd.read_csv('Data/movies.csv')
df_ratings=pd.read_csv('Data/ratings.csv')
df_tags=pd.read_csv('Data/tags.csv')

#### Understanding the Data

In [ ]:
# Explore the links dataset
df_links.head()  # show first few rows

In [ ]:
print(df_links.info())          # Get info on data types and non-null counts
print(df_links.describe())     # Summary statistics for numerical columns
print(df_links.isnull().sum())  # Check for missing values


In [ ]:
# Explore the movies dataset
df_movies.head()  # show first few rows

In [ ]:
print(df_movies.info())          # Get info on data types and non-null counts
print(df_movies.describe())     # Summary statistics for numerical columns
print(df_movies.isnull().sum())  # Check for missing values

In [ ]:
# Explore the ratings dataset
df_ratings.head()  # show first few rows

In [ ]:
print(df_ratings.info())         # Get info on data types and non-null counts
print(df_ratings.describe())     # Summary statistics for numerical columns
print(df_ratings.isnull().sum()) # Check for missing values

In [ ]:
# Explore the tags dataset
df_tags.head()  # show first few rows

In [ ]:
print(df_tags.info())         # Get info on data types and non-null counts
print(df_tags.describe())     # Summary statistics for numerical columns
print(df_tags.isnull().sum()) # Check for missing values

#### Handle missing data

In [83]:
df_links = df_links.dropna() # drop missing rows

In [ ]:
df_links.isnull().sum() #check the missing data again

In [ ]:
df_links.describe()

#### Check for duplicates

In [ ]:
# Duplicates in links.csv
df_links.duplicated().sum()

In [ ]:
# Duplicates in movies.csv
df_movies.duplicated().sum()

In [ ]:
# Duplicates in ratings.csv
df_ratings.duplicated().sum()

In [ ]:
# Duplicates in tags.csv
df_tags.duplicated().sum()

#### Convert data to correct data types

In [90]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'])    # Convert to datetime
df_tags['timestamp'] = pd.to_datetime(df_tags['timestamp'])

In [ ]:
df_tags.head()

#### Handling outliers

In [ ]:
from scipy import stats

# Calculate the Z-scores for the column
z_scores = np.abs(stats.zscore(df_ratings['rating']))

# Set a threshold for identifying outliers
threshold = 3

# Find rows where Z-score is greater than the threshold
outliers = df_ratings[z_scores > threshold]
print(outliers)

No outliers detected

#### Deal with Movies.csv genre column

In [ ]:
df_movies['genres'] = df_movies['genres'].str.split('|')
df_exploded_movies = df_movies.explode('genres')
df_exploded_movies

#### One-hot encoding

In [ ]:
# Pivot to create a one-hot encoded DataFrame for genres
genres_encoding = df_exploded_movies.pivot_table(index='movieId', columns='genres', aggfunc=lambda x: 1, fill_value=0)

# Flatten the multi-level column index resulting from pivot to get simple column names
genres_encoding.columns = genres_encoding.columns.get_level_values(1)
genres_encoding = genres_encoding.add_prefix('genre_')

# Merge the one-hot encoded genres with the original movies DataFrame 
df_movies_encoded = pd.merge(df_movies.drop(columns='genres').drop_duplicates(), genres_encoding, on='movieId', how='left')

df_movies_encoded

#### Merge the data

In [ ]:
df_ratings_movies_merged= pd.merge(df_movies_encoded, df_ratings, on='movieId', how='inner') # merge ratings.csv with movies.csv
df_ratings_movies_merged

In [ ]:
df_merged=pd.merge(df_ratings_movies_merged, df_tags, on=['movieId', 'userId'], how='left')
df_merged

#### Deal with the NaN values in tags

In [ ]:
# Create a binary indicator column for the presence of tags
df_merged['tag_present'] = df_merged['tag'].notna().astype(int)

df_merged.head()

In [103]:
# Identify duplicates in userId and movieId pairs
duplicates = df_merged[df_merged.duplicated(subset=['userId', 'movieId'], keep=False)]
print(f"Number of duplicate entries: {len(duplicates)}")

Number of duplicate entries: 2412


In [104]:
# Remove duplicates, keeping the first occurrence
df_merged_cleaned = df_merged.drop_duplicates(subset=['userId', 'movieId'], keep='first')

In [105]:
# Now, create the user-item matrix
user_item_matrix = df_merged_cleaned.pivot(index='userId', columns='movieId', values='rating')

In [106]:
# Mean-center the user-item matrix by subtracting each user's average rating
user_item_matrix_mean_centered = user_item_matrix.sub(user_item_matrix.mean(axis=1), axis=0)

# Display the first few rows of the mean-centered matrix to confirm
user_item_matrix_mean_centered.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,NaN,-0.366379,NaN,NaN,-0.366379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# Check for duplicates again in the cleaned DataFrame to confirm removal
duplicates_after_cleaning = df_merged_cleaned[df_merged_cleaned.duplicated(subset=['userId', 'movieId'], keep=False)]
num_duplicates_after = len(duplicates_after_cleaning)

# Display result to confirm duplicates have been removed
num_duplicates_after

0